In [4]:
import pandas as pd
df = pd.read_json('../training.json')
df.head()

,title,entity,text,published,nasdaq_label,nyse_label,stock_published,+1day,stock_+1day,+1week,stock_+1week,+1month,stock_+1month,+3month,stock_+3month,+6month,stock_+6month
0,ANZ Bank New Zealand sells life insurance busi...,cigna corp,May 30 (Reuters) - ANZ Bank New Zealand on Wed...,1527637320000,None,"[CI, cigna corp, 0.0]",173.320007,1527723720000,169.369995,1528242120000,174.929993,1530315720000,169.949997,1535586120000,189.800003,1543534920000,213.130005
1,Australia's Westpac cleared of rate rigging bu...,westpac banking corp,SYDNEY (Reuters) - Australia’s Westpac Banking...,1527138840000,None,"[WBK, westpac banking corp, 0.0]",21.459999,1527225240000,21.309999,1527743640000,21.139999,1529817240000,21.799999,1535087640000,20.110001,1543036440000,19.070000
2,"Clear Channel Outdoor Holdings, Inc. Reports R...","clear channel outdoor holdings, inc.",SAN ANTONIO--(BUSINESS WIRE)-- Clear Channel O...,1526987700000,None,"[CCO, clear channel outdoor holdings, 0.0]",4.500000,1527074100000,4.450000,1527592500000,4.400000,1529666100000,4.650000,1534936500000,4.700000,1542885300000,5.860000
3,"Clear Channel Outdoor Holdings, Inc. Reports R...","clear channel outdoor holdings, inc",SAN ANTONIO--(BUSINESS WIRE)-- Clear Channel O...,1526987700000,None,"[CCO, clear channel outdoor holdings, 0.0]",4.500000,1527074100000,4.450000,1527592500000,4.400000,1529666100000,4.650000,1534936500000,4.700000,1542885300000,5.860000
4,BRIEF-Pembina Pipeline Corp Reports Q1 EPS $0.59,brief-pembina pipeline corp,May 3 (Reuters) - Pembina Pipeline Corp:\n* OR...,1525387380000,None,"[PBA, pembina pipeline cor, 0.0]",33.939999,1525473780000,33.939999,1525992180000,34.650002,1528065780000,34.459999,1533336180000,36.590000,1541284980000,33.070000


In [20]:
time_periods = [
    'published',
    '+1day',
    '+1week',
    '+1month',
    '+3month',
    '+6month'
]

train_df = df.drop(columns = time_periods)
train_df = train_df.drop(columns = ['entity','nasdaq_label','nyse_label'])


In [50]:
observations = [
    '+1day',
    '+1week',
    '+1month',
    '+3month',
    '+6month'
]
def build_train_df1(source_df):
    train_df1 = pd.DataFrame(train_df['title'])
    for observation in observations:
        train_df1[f'change_{observation}'] = (train_df[f'stock_{observation}'] / train_df['stock_published']) - 1
    return train_df1

In [35]:
train_df[train_df['stock_published'] == 0]
# No division by zero

,title,text,stock_published,stock_+1day,stock_+1week,stock_+1month,stock_+3month,stock_+6month


In [37]:
train_df = train_df[train_df['stock_published'].notna()]

In [45]:
import math
train_df[train_df.apply(
    lambda row: any(
        map(
            lambda observation: math.isnan(row.loc[f'stock_{observation}']),
            observations
        )
    ),
    axis=1
)]
# All NaN values shold be dropped

,title,text,stock_published,stock_+1day,stock_+1week,stock_+1month,stock_+3month,stock_+6month


In [48]:
train_df1 = build_train_df1(train_df)

NameError: name 'build_train_df1' is not defined